In [1]:
import numpy as np
np.random.seed(71)

import matplotlib
matplotlib.use('Agg')

from keras import backend as K
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.datasets import cifar10

import matplotlib.pyplot as plt


import multiprocessing as mp
import math

Using TensorFlow backend.


In [0]:
def Hbeta(D, beta):
  P = np.exp(-D * beta)
  sumP = np.sum(P) + 10e-15
  
  H = np.log(sumP) + beta * np.sum(D * P) / sumP
  P = P / sumP
  return H, P

In [0]:
def x2p_job(data):
    i, Di, tol, logU = data
    beta = 1.0
    betamin = -np.inf
    betamax = np.inf
    H, thisP = Hbeta(Di, beta)

    Hdiff = H - logU
    tries = 0
    while np.abs(Hdiff) > tol and tries < 50:
        if Hdiff > 0:
            betamin = beta
            if betamax == -np.inf:
                beta = beta * 2
            else:
                beta = (betamin + betamax) / 2
        else:
            betamax = beta
            if betamin == -np.inf:
                beta = beta / 2
            else:
                beta = (betamin + betamax) / 2

        H, thisP = Hbeta(Di, beta)
        Hdiff = H - logU
        tries += 1

    return i, thisP

In [0]:
def x2p(X):
    tol = 1e-5
    n = X.shape[0]
    logU = np.log(perplexity)

    sum_X = np.sum(np.square(X), axis=1)
    D = sum_X + (sum_X.reshape([-1, 1]) - 2 * np.dot(X, X.T))

    idx = (1 - np.eye(n)).astype(bool)
    D = D[idx].reshape([n, -1])

    def generator():
        for i in range(n):
            yield i, D[i], tol, logU

    pool = mp.Pool(n_jobs)
    result = pool.map(x2p_job, generator())
    P = np.zeros([n, n])
    for i, thisP in result:
        P[i, idx[i]] = thisP

    return P

In [0]:
def calculate_P(X):
    n = X.shape[0]
    P = np.zeros([n, batch_size])
    for i in range(0, n, batch_size):
        P_batch = x2p(X[i:i + batch_size])
        P_batch[np.isnan(P_batch)] = 0
        P_batch = P_batch + P_batch.T
        P_batch = P_batch / P_batch.sum()
        P_batch = np.maximum(P_batch, 1e-12)
        P[i:i + batch_size] = P_batch
    return P

In [0]:
def KLdivergence(P, Y):
    low_dim = 2
    alpha = low_dim - 1.
    sum_Y = K.sum(K.square(Y), axis=1)
    eps = K.variable(10e-15)
    D = sum_Y + K.reshape(sum_Y, [-1, 1]) - 2 * K.dot(Y, K.transpose(Y))
    Q = K.pow(1 + D / alpha, -(alpha + 1) / 2)
    Q *= K.variable(1 - np.eye(batch_size))
    Q /= K.sum(Q)
    Q = K.maximum(Q, eps)
    C = K.log(P) - K.log(Q)
    C = K.sum(P * C)
    return C

In [0]:
def LoadData():
  print ("Loading cifar-10")
  (X_train, y_train), (X_test, y_test) = cifar10.load_data()

  dataset = (X_train, y_train), (X_test, y_test)
  return dataset

In [0]:
def Reshape(dataset):
  (X_train, y_train), (X_test, y_test) = dataset
  n, channel, row, col = X_train.shape
  X_train = X_train.reshape(-1, channel * row * col)
  X_test = X_test.reshape(-1, channel * row * col)
  
  print ("X_train.shape:", X_train.shape)
  print ("X_test.shape:", X_test.shape)

  dataset = (X_train, y_train), (X_test, y_test), n
  return dataset

In [0]:
def Normalize(dataset):
  (X_train, y_train), (X_test, y_test), n = dataset
  print ("Max in training data before scaling:", str(np.max(X_train)))
  
  X_train = X_train.astype('float32')
  X_test = X_test.astype('float32')
  X_train /= 255
  X_test /= 255
  minimum = np.min(X_train)
  maximum = np.max(y_train)
  
  print ("Max in training data after scaling:", str(np.max(X_train)))
  
  dataset = (X_train, y_train), (X_test, y_test), n
  return dataset

In [0]:
def GenerateBatch(batch_size,nb_epoch):
  for epoch in range(nb_epoch):
    for i in range(0, n, batch_size):
      yield(X_train[i:i+batch_size], calculate_P(X_train[i:i+batch_size]))


In [0]:
# Plot the training and validation loss + accuracy
def plot_training(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    epochs = range(len(acc))

    plt.plot(epochs, acc, 'r.')
    plt.plot(epochs, val_acc, 'r')
    plt.title('Training and validation accuracy')

    plt.show()

    plt.savefig('acc_vs_epochs.png')


In [12]:
(X_train, y_train), (X_test, y_test), n  = Normalize(Reshape(LoadData()))
n = 10000
X_train = X_train[0:n]
y_train = y_train[0:n]


Loading cifar-10
X_train.shape: (50000, 3072)
X_test.shape: (10000, 3072)
Max in training data before scaling: 255
Max in training data after scaling: 1.0


In [13]:
perplexity = 30.0
n_jobs = 5
nb_epoch = 2
batch_size = 100


# build model
model = Sequential()
model.add(Dense(20, input_shape=(X_train.shape[1],),activation='relu'))
model.add(Dense(40, activation='relu'))
model.add(Dense(2))
model.compile(loss=KLdivergence, optimizer="adam")


# fit model
history = model.fit_generator(GenerateBatch(batch_size,nb_epoch),steps_per_epoch=math.ceil(n/batch_size),epochs=nb_epoch, shuffle = True)


W0812 01:09:23.206142 139712706385792 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0812 01:09:23.221236 139712706385792 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0812 01:09:23.223717 139712706385792 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0812 01:09:23.259864 139712706385792 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0812 01:09:23.293410 139712706385792 deprecation_wrappe

Epoch 1/2
100/100 [==============================] - 25s 254ms/step - loss: 0.5073 - val_loss: nan
Epoch 2/2
100/100 [==============================] - 47s 468ms/step - loss: 0.4797 - val_loss: nan


In [0]:

# plot 
fig = plt.figure(figsize=(5, 5))
pred = model.predict(X_test[0:100])
plt.scatter(pred[:, 0], pred[:, 1], marker='o', s=4, edgecolor='')
fig.tight_layout()

In [0]:
# plot 
acc = history.history['loss']
epochs = range(len(acc))


plt.plot(epochs, acc, 'r.')

plt.title('Training and validation accuracy')

plt.show()

plt.savefig('acc_vs_epochs.png')

In [16]:
history.history

{'loss': [0.5072732537984848, 0.479733766913414], 'val_loss': [nan, nan]}